DGM

In [107]:
import os
import yaml
import torch
import pickle
import numpy as np
from models.mart import MART

In [108]:
def load_config(config_path):
    """
    Load configuration from a YAML file.

    Args:
        config_path (str): Path to the YAML configuration file.

    Returns:
        dict: Loaded configuration dictionary.
    """
    print(f"[INFO] Loading configuration from {config_path}...")
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    print("[INFO] Configuration loaded successfully!")
    return config


In [109]:
CHECKPOINT_PATH = "files/sdd_ckpt_best.pth"  # Update this
TEST_DATA_PATH = "files/sdd_test.pkl"  # Update this
CONFIG_PATH = "files/mart_sdd_reproduce.yaml"  # Update this
BATCH_SIZE = 64

In [110]:
def load_model(checkpoint_path, opts):
    print("[INFO] Loading model from checkpoint...")
    args = SimpleNamespace(**opts)
    model = MART(args)  # Remove .cuda()
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'])  # or whatever loading method you are using

    print("[INFO] Model loaded successfully!")
    return model


In [111]:
# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model onto the correct device
model = model.to(device)

def evaluate_model(model, test_loader, opts):
    print("[INFO] Starting evaluation...")

    ade_sum, fde_sum, total_agents = 0.0, 0.0, 0

    with torch.no_grad():
        for sample in test_loader:
            x_abs, y = sample
            x_abs, y = x_abs.to(device), y.to(device)  # Ensure data is on the same device as the model

            batch_size, num_agents, length, _ = x_abs.size()

            x_rel = torch.zeros_like(x_abs)
            x_rel[:, :, 1:] = x_abs[:, :, 1:] - x_abs[:, :, :-1]
            x_rel[:, :, 0] = x_rel[:, :, 1]

            y_pred = model(x_abs, x_rel)

            if opts["pred_rel"]:
                cur_pos = x_abs[:, :, [-1]].unsqueeze(2)
                y_pred = torch.cumsum(y_pred, dim=3) + cur_pos

            ade = torch.min(torch.mean(torch.norm(y_pred - y[:, :, None], dim=-1), dim=3), dim=2)[0].mean().item()
            fde = torch.min(torch.mean(torch.norm(y_pred[:, :, :, -1:] - y[:, :, None, -1:], dim=-1), dim=3), dim=2)[0].mean().item()

            ade_sum += ade * num_agents * batch_size
            fde_sum += fde * num_agents * batch_size
            total_agents += num_agents * batch_size

    ade_avg = (ade_sum / total_agents) * opts["scale"]
    fde_avg = (fde_sum / total_agents) * opts["scale"]

    print(f"[INFO] Evaluation Results: ADE = {ade_avg:.4f}, FDE = {fde_avg:.4f}")
    return ade_avg, fde_avg


In [112]:
import torch
import numpy as np

class TrajectoryDataset(Dataset):
    def __init__(
        self, obs_len=8, pred_len=12, mode='train', scale=10, inputs=None, max_agents=50
    ):
        super(TrajectoryDataset, self).__init__()

        self.obs_len = obs_len
        self.pred_len = pred_len
        self.seq_len = self.obs_len + self.pred_len
        self.scale = scale
        self.max_agents = max_agents

        with open('files/sdd_test.pkl'.format(mode), 'rb') as f:
            traj = pickle.load(f)

        traj_tmp = []

        for t in traj:
            traj_tmp.append(t)
            if mode == 'train':
                traj_tmp.append(np.flip(t, axis=1))

        self.traj = []
        if 'pos_x' in inputs and 'pos_y' in inputs:
            for t in traj_tmp:
                t -= t[:, :1, :]
                self.traj.append(t)
        else:
            self.traj = traj_tmp


    def __len__(self):
        return len(self.traj)

    def __getitem__(self, index):
        past_traj = self.traj[index][:, :self.obs_len] * self.scale
        future_traj = self.traj[index][:, self.obs_len:] * self.scale

        # Padding the number of agents to max_agents if necessary
        num_agents = past_traj.shape[0]
        if num_agents < self.max_agents:
            pad_size = self.max_agents - num_agents
            past_traj_padded = np.pad(past_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
            future_traj_padded = np.pad(future_traj, ((0, pad_size), (0, 0), (0, 0)), mode='constant')
        else:
            past_traj_padded = past_traj
            future_traj_padded = future_traj

        past_traj = torch.from_numpy(past_traj_padded).type(torch.float)
        future_traj = torch.from_numpy(future_traj_padded).type(torch.float)

        return [past_traj, future_traj]


In [113]:
from torch.utils.data import DataLoader
if __name__ == "__main__":
    opts = load_config(CONFIG_PATH)
    model = load_model(CHECKPOINT_PATH, opts)

    # Prepare the test dataset and DataLoader
    test_dataset = TrajectoryDataset(
        obs_len=opts["past_length"],
        pred_len=opts["future_length"],
        mode="test",  # This loads the test set
        scale=opts["scale"],
        inputs=opts["inputs"]
    )
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    print("[INFO] Starting evaluation with test data...")
    evaluate_model(model, test_loader, opts)

[INFO] Loading configuration from files/mart_sdd_reproduce.yaml...
[INFO] Configuration loaded successfully!
[INFO] Loading model from checkpoint...
[INFO] PRT Agg: cat
[INFO] HRT Agg: avg
[INFO] Binary Threshold Function Type: 2
[INFO] Model loaded successfully!
[INFO] Starting evaluation with test data...
[INFO] Starting evaluation...


<ipython-input-110-b8c79bf0e098>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location='cpu')


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import torch
torch.cuda.is_available()